In [ ]:
!pip install langchain openai tiktoken selenium unstructured faiss-cpu -q



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
urls = {
    "https://indiankanoon.org/doc/62271433",
    "https://indiankanoon.org/doc/99382937",
}

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
data[1]

Document(page_content="Premium Members\n\nAdvanced Search\n\nCase Removal\n\nWarning on Translation\n\nTake notes as you read a judgment using our Virtual Legal Assistant and get email alerts whenever a new judgment matches your query (Query Alert Service). Try out our Premium Member services -- Free for one month.\n\nTripura High Court\n\nRamesh Debbarma vs The State Of Tripura And Others on 22 September, 2023\n\nPage 1 of 2\n\n\n\n\n                        HIGH COURT OF TRIPURA\n                              AGARTALA\n                            WP(C) No.603 of 2023\nRamesh Debbarma\n                                                         ...... Petitioner(s)\n                            VERSUS\nThe State of Tripura and others\n                                                       ...... Respondent(s)\n\nFor Petitioner(s)               :      Mr. P. Roy Barman, Senior Advocate.\n\nMr. K. Chakraborty, Advocate.\n\nFor Respondent(s)               :      Mr. D. Sarkar, Advocate.\n\nHO

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)


docs = text_splitter.split_documents(data)

In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.3
                 , model="gpt-3.5-turbo")

In [ ]:
! pip install faiss-cpu

In [ ]:
db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [ ]:
doc_retriever= db.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA
from langchain.memory import ConversationKGMemory, ReadOnlySharedMemory

memory = ConversationKGMemory(memory_key="chat_history",llm=llm)
readonlymemory = ReadOnlySharedMemory(memory=memory)

qa= RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_retriever,memory=readonlymemory)

In [ ]:
qa.run("who is the defender for  Ramesh Debbarma vs The State Of Tripura case ")

'The defender for the case Ramesh Debbarma vs The State Of Tripura is Mr. D. Sarkar, Advocate.'